In [1]:
from glob import glob
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle, ScipyParticle
from parcels import ErrorCode, AdvectionRK4_3D, Variable, Field
from parcels.application_kernels.TEOSseawaterdensity import PolyTEOS10_bsq

from datetime import timedelta
from datetime import datetime
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
# Lorenz - MOi fields
data_path = '/storage/shared/oceanparcels/input_data/MOi/2019/'

output_path = '/storage/shared/oceanparcels/output_data/' + \
    'data_Claudio/v_s_RK4.nc'

ufiles = []
vfiles = []
wfiles = []
tfiles = []
sfiles = [] 
twoDfiles = []

for i in range(8, 10):
    ufiles = ufiles + sorted(glob(data_path + f'psy4v3r1-daily_U_201{i}*.nc'))
    vfiles = vfiles + sorted(glob(data_path + f'psy4v3r1-daily_V_201{i}*.nc'))
    wfiles = wfiles + sorted(glob(data_path + f'psy4v3r1-daily_W_201{i}*.nc'))
    tfiles = tfiles + sorted(glob(data_path + f'psy4v3r1-daily_T_201{i}*.nc'))    
    sfiles = sfiles + sorted(glob(data_path + f'psy4v3r1-daily_S_201{i}*.nc'))
    twoDfiles = twoDfiles + sorted(glob(data_path + f'psy4v3r1-daily_2D_201{i}*.nc'))

# phfiles = sorted(glob(bio_data_path + 'biomer4v2r1-weekly_ph_2019*.nc'))

mesh_mask = '/storage/shared/oceanparcels/input_data/MOi/domain_ORCA0083-N006/coordinates.nc'
bathy_file = '/storage/shared/oceanparcels/input_data/MOi/domain_ORCA0083-N006/bathymetry_ORCA12_V3.3.nc'
# mesh_mask_bio = '/storage/shared/oceanparcels/input_data/MOi/domain_ORCA025-N006/coordinates.nc'


In [3]:
filenames = {'U': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': ufiles},
             'V': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': vfiles},
             'W': {'lon': mesh_mask,
                   'lat': mesh_mask,
                   'depth': wfiles[0],
                   'data': wfiles}}

filenames['cons_temperature'] = {'lon': mesh_mask, 
                                 'lat': mesh_mask, 
                                 'depth': wfiles[0], 
                                 'data': tfiles}
filenames['abs_salinity'] = {'lon': mesh_mask, 
                             'lat': mesh_mask, 
                             'depth': wfiles[0], 
                             'data': sfiles}

filenames['mld'] = {'lon': mesh_mask, 
                     'lat': mesh_mask,
                     'depth': twoDfiles[0],
                     'data': twoDfiles}


In [4]:
n_points = 100
particle_size = 0.0001 #
initial_depth = 5

start_time = datetime.strptime('2019-12-20 12:00:00', '%Y-%m-%d %H:%M:%S')
# psy4v3r1-daily_2D_2019-01-01.nc

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz'}

variables['cons_temperature'] = 'votemper'
variables['abs_salinity'] = 'vosaline'
variables['mld'] = 'somxlavt'

dimensions = {'U': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'},
              'V': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'},
              'W': {'lon': 'glamf',
                    'lat': 'gphif',
                    'depth': 'depthw',
                    'time': 'time_counter'}}

dimensions['cons_temperature'] = {'lon': 'glamf', 
                                  'lat': 'gphif',
                                  'depth': 'depthw', 
                                  'time': 'time_counter'}

dimensions['abs_salinity'] = {'lon': 'glamf', 
                              'lat': 'gphif',
                              'depth': 'depthw', 
                              'time': 'time_counter'}

dimensions['mld'] = {'lon': 'glamf', 
                              'lat': 'gphif',
                              'depth': 'deptht', 
                              'time': 'time_counter'}



In [5]:
indices = {'lat': range(750, 1300), 'lon': range(2900, 4000)}

In [6]:
fieldset = FieldSet.from_nemo(filenames, variables, dimensions,
                              allow_time_extrapolation=False,
                              indices=indices)


         It will be opened with no decoding. Filling values might be wrongly parsed.


In [7]:
fieldset.add_constant('viscosity', 1e-6)
fieldset.add_constant('particle_density', 1380)

bathy = xr.load_dataset(bathy_file)

fieldset.add_field(Field('bathymetry', bathy['Bathymetry'].values,
                         lon=bathy['nav_lon'].values, lat=bathy['nav_lat'].values,
                         mesh='spherical'))


In [8]:
class PlasticParticle(ScipyParticle):
    cons_temperature = Variable('cons_temperature', dtype=np.float32, initial=fieldset.cons_temperature)
    abs_salinity = Variable('abs_salinity', dtype=np.float32, initial=fieldset.abs_salinity)
    mld = Variable('mld', dtype=np.float32, initial=fieldset.mld)
    density = Variable('density', dtype=np.float32, initial=1035)
    v_s = Variable('v_s', dtype=np.float32, initial=0)
    alpha = Variable('alpha', dtype=np.float32, initial=particle_size)
    

In [9]:
lon_cluster = [6.287]*n_points
lat_cluster = [-32.171]*n_points

# depth_cluster = np.linspace(1, 5000, n_points)
depth_cluster = np.ones(n_points)*initial_depth #meters

date_cluster = [start_time]*n_points

lon_cluster = np.array(lon_cluster)+(np.random.random(len(lon_cluster))-0.5)/24
lat_cluster = np.array(lat_cluster)+(np.random.random(len(lat_cluster))-0.5)/24

In [10]:
pset = ParticleSet.from_list(fieldset=fieldset, pclass=PlasticParticle,
                             lon=lon_cluster,
                             lat=lat_cluster,
                             depth=depth_cluster,
                             time=date_cluster)


In [11]:
def delete_particle(particle, fieldset, time):
    particle.delete()

In [12]:
def GrowParticle(particle, fieldset, time):
    particle.alpha = particle.alpha - fieldset.grow_rate*particle.dt

In [13]:
def AdvectionRK4_1D(particle, fieldset, time):
    """
    Advection of particles using fourth-order Runge-Kutta integration including vertical velocity.
    Function needs to be converted to Kernel object before execution
    """
    (w1) = fieldset.W[time, particle.depth, particle.lat, particle.lon]
    depth_1 = particle.depth + w1 * .5 * particle.dt
    
    (w2) = fieldset.W[time + .5 * particle.dt, depth_1, particle.lat, particle.lon]
    depth_2 = particle.depth + w2 * .5 * particle.dt

    (w3) = fieldset.W[time + .5 * particle.dt, depth_2, particle.lat, particle.lon]
    depth_3 = particle.depth + w3 * particle.dt
    
    (w4) = fieldset.W[time + particle.dt, depth_3, particle.lat, particle.lon]

    particle.depth += (w1 + 2 * w2 + 2 * w3 + w4) / 6. * particle.dt

In [14]:
def SinkingVelocity_RK4(particle, fieldset, time):
    def polyTEOS10_bsq(Z, SA, CT):
        Z = - Z #particle.depth  # note: use negative depths!

        SAu = 40 * 35.16504 / 35
        CTu = 40
        Zu = 1e4
        deltaS = 32
        R000 = 8.0189615746e+02
        R100 = 8.6672408165e+02
        R200 = -1.7864682637e+03
        R300 = 2.0375295546e+03
        R400 = -1.2849161071e+03
        R500 = 4.3227585684e+02
        R600 = -6.0579916612e+01
        R010 = 2.6010145068e+01
        R110 = -6.5281885265e+01
        R210 = 8.1770425108e+01
        R310 = -5.6888046321e+01
        R410 = 1.7681814114e+01
        R510 = -1.9193502195e+00
        R020 = -3.7074170417e+01
        R120 = 6.1548258127e+01
        R220 = -6.0362551501e+01
        R320 = 2.9130021253e+01
        R420 = -5.4723692739e+00
        R030 = 2.1661789529e+01
        R130 = -3.3449108469e+01
        R230 = 1.9717078466e+01
        R330 = -3.1742946532e+00
        R040 = -8.3627885467e+00
        R140 = 1.1311538584e+01
        R240 = -5.3563304045e+00
        R050 = 5.4048723791e-01
        R150 = 4.8169980163e-01
        R060 = -1.9083568888e-01
        R001 = 1.9681925209e+01
        R101 = -4.2549998214e+01
        R201 = 5.0774768218e+01
        R301 = -3.0938076334e+01
        R401 = 6.6051753097e+00
        R011 = -1.3336301113e+01
        R111 = -4.4870114575e+00
        R211 = 5.0042598061e+00
        R311 = -6.5399043664e-01
        R021 = 6.7080479603e+00
        R121 = 3.5063081279e+00
        R221 = -1.8795372996e+00
        R031 = -2.4649669534e+00
        R131 = -5.5077101279e-01
        R041 = 5.5927935970e-01
        R002 = 2.0660924175e+00
        R102 = -4.9527603989e+00
        R202 = 2.5019633244e+00
        R012 = 2.0564311499e+00
        R112 = -2.1311365518e-01
        R022 = -1.2419983026e+00
        R003 = -2.3342758797e-02
        R103 = -1.8507636718e-02
        R013 = 3.7969820455e-01
        ss = math.sqrt((SA + deltaS) / SAu)
        tt = CT / CTu
        zz = -Z / Zu
        rz3 = R013 * tt + R103 * ss + R003
        rz2 = (R022 * tt + R112 * ss + R012) * tt + (R202 * ss + R102) * ss + R002
        rz1 = (((R041 * tt + R131 * ss + R031) * tt + (R221 * ss + R121) * ss + R021) * tt + ((R311 * ss + R211) * ss + R111) * ss + R011) * tt + (((R401 * ss + R301) * ss + R201) * ss + R101) * ss + R001
        rz0 = (((((R060 * tt + R150 * ss + R050) * tt + (R240 * ss + R140) * ss + R040) * tt + ((R330 * ss + R230) * ss + R130) * ss + R030) * tt + (((R420 * ss + R320) * ss + R220) * ss + R120) * ss + R020) * tt + ((((R510 * ss + R410) * ss + R310) * ss + R210) * ss + R110) * ss + R010) * tt + (((((R600 * ss + R500) * ss + R400) * ss + R300) * ss + R200) * ss + R100) * ss + R000
        density = ((rz3 * zz + rz2) * zz + rz1) * zz + rz0

        return density
    
    g = 9.81
    dt = particle.dt
    rho_p = fieldset.particle_density
    
    nu = fieldset.viscosity
    alpha = particle.alpha
    seafloor = fieldset.bathymetry[time, particle.depth,
                                   particle.lat, particle.lon]
    
    if particle.depth < seafloor and particle.depth > 0:
        T1 = fieldset.cons_temperature[time, particle.depth,
                                  particle.lat, particle.lon]
        S1 = fieldset.abs_salinity[time, particle.depth,
                                 particle.lat, particle.lon]
        depth_1 = particle.depth
        # --
        rho_f = polyTEOS10_bsq(depth_1, S1, T1)
        beta = 3*rho_f/(2*rho_p + rho_f)
        tau_p = alpha*alpha/(3*beta*nu)
        v_s_1 = (1 - beta)*g*tau_p
        # --
        depth_2 = depth_1 + v_s_1*dt*0.5
        T2 = fieldset.cons_temperature[time + .5*dt, depth_1,
                                  particle.lat, particle.lon]
        S2 = fieldset.abs_salinity[time + .5*dt, depth_1,
                                 particle.lat, particle.lon]
        rho_f = polyTEOS10_bsq(depth_1, S2, T2)
        beta = 3*rho_f/(2*rho_p + rho_f)
        tau_p = alpha*alpha/(3*beta*nu) ## alpha*alpha
        v_s_2 = (1 - beta)*g*tau_p
        # --
        depth_3 = depth_2 + v_s_2*dt*0.5
        T3 = fieldset.cons_temperature[time + .5*dt, depth_2,
                                  particle.lat, particle.lon]
        S3 = fieldset.abs_salinity[time + .5*dt, depth_2,
                                 particle.lat, particle.lon]
        rho_f = polyTEOS10_bsq(depth_3, S3, T3)
        beta = 3*rho_f/(2*rho_p + rho_f)
        tau_p = alpha*alpha/(3*beta*nu) ## alpha*alpha
        v_s_3 = (1 - beta)*g*tau_p
        # --
        depth_4 = depth_3 + v_s_3*dt
        T4 = fieldset.cons_temperature[time + dt, depth_3,
                                  particle.lat, particle.lon]
        S4 = fieldset.abs_salinity[time + dt, depth_3,
                                 particle.lat, particle.lon]
        rho_f = polyTEOS10_bsq(depth_4, S4, T4)
        beta = 3*rho_f/(2*rho_p + rho_f)
        tau_p = alpha*alpha/(3*beta*nu) ## alpha*alpha
        v_s_4 = (1 - beta)*g*tau_p
    
        v_s = (v_s_1 + 2 * v_s_2 + 2 * v_s_3 + v_s_4)/6.
        
    else:
        v_s = 0
        
    particle.depth = particle.depth + v_s*dt

In [15]:
def SinkingVelocity(particle, fieldset, time):
    rho_p = fieldset.particle_density ##
    rho_f = particle.density
    nu = fieldset.viscosity
    alpha = particle.alpha
    g = 9.81
    dt = particle.dt
    beta = 3*rho_f/(2*rho_p + rho_f)
    tau_p = alpha*alpha/(3*beta*nu) ## alpha*alpha
    
    seafloor = fieldset.bathymetry[time, particle.depth, particle.lat, particle.lon]
    
    if particle.depth < seafloor and particle.depth > 0:
        v_s = (1 - beta)*g*tau_p
    else:
        v_s = 0
        
    particle.v_s = v_s
    particle.depth = particle.depth + v_s*dt
    

In [16]:
def SampleField(particle, fielset, time):
    particle.cons_temperature = fieldset.cons_temperature[time, particle.depth, 
                                               particle.lat, particle.lon]
    particle.abs_salinity = fieldset.abs_salinity[time, particle.depth, 
                                               particle.lat, particle.lon]
    particle.mld = fieldset.mld[time, particle.depth, 
                                               particle.lat, particle.lon]


In [17]:
kernels = pset.Kernel(AdvectionRK4_1D) + pset.Kernel(SampleField) + pset.Kernel(PolyTEOS10_bsq) + pset.Kernel(SinkingVelocity_RK4)

# Output file
output_file = pset.ParticleFile(name=output_path,
                                outputdt=timedelta(hours=24))

pset.execute(kernels,
             output_file=output_file,
             runtime=timedelta(days=10),
             dt=-timedelta(hours=1),
             recovery={ErrorCode.ErrorOutOfBounds: delete_particle})

output_file.close()

INFO: Temporary output files are stored in /storage/shared/oceanparcels/output_data/data_Claudio/out-GBVYQWSN.
INFO: You can use "parcels_convert_npydir_to_netcdf /storage/shared/oceanparcels/output_data/data_Claudio/out-GBVYQWSN" to convert these to a NetCDF file during the run.
N/A% (0 of 864000.0) |                   | Elapsed Time: 0:00:00 ETA:  --:--:--WARNING: ParticleSet is empty on writing as array at time 6.18624e+07
 20% (172800.0 of 864000.0) |##          | Elapsed Time: 0:00:16 ETA:   0:01:05WARNING: ParticleSet is empty on writing as array at time 6.1776e+07
 30% (259200.0 of 864000.0) |###         | Elapsed Time: 0:00:27 ETA:   0:01:16WARNING: ParticleSet is empty on writing as array at time 6.16896e+07
 40% (345600.0 of 864000.0) |####        | Elapsed Time: 0:00:45 ETA:   0:01:48WARNING: ParticleSet is empty on writing as array at time 6.16032e+07
 50% (432000.0 of 864000.0) |######      | Elapsed Time: 0:01:03 ETA:   0:01:28WARNING: ParticleSet is empty on writing as a

-----------
Analysis

In [ ]:
sim = xr.load_dataset(output_path)

In [ ]:
sim

In [ ]:
sim['z']

In [ ]:
plt.plot(sim['alpha'][0, :])
plt.xlabel('Time steps')
plt.ylabel('Particle size')

In [ ]:
step = 10
plt.scatter(sim['alpha'][:,step], -sim['z'][:,step])
plt.xlabel('Salinity (PSU)')
plt.ylabel('Depth (m)')
plt.title(sim['time'][0,step].values)

In [ ]:
step = 10
plt.scatter(sim['salinity'][:,step], -sim['z'][:,step])
plt.xlabel('Salinity (PSU)')
plt.ylabel('Depth (m)')
plt.title(sim['time'][0,step].values)

In [ ]:
step = 10
plt.scatter(sim['temperature'][:,step], -sim['z'][:,step], color='r')
plt.xlabel('Temperature (Celsius)')
plt.ylabel('Depth (m)')
plt.title(sim['time'][0,step].values)

In [ ]:
step = 10
plt.scatter(sim['ph'][:,step], -sim['z'][:,step], color='g')
plt.xlabel('pH')
plt.ylabel('Depth (m)')
plt.title(sim['time'][0,step].values)

In [ ]:
step = 10
for i in range(1000):
    plt.plot(sim['lon'][i,:], sim['lat'][i,:]);

plt.xlabel('Longitude')
plt.ylabel('Latitude')
# plt.title(sim['time'][0,step].values)

In [ ]:
sim2 = xr.load_dataset('/storage/shared/oceanparcels/output_data/data_Claudio/SA_5m_s02.nc')

In [ ]:
sim2

In [ ]:
import cartopy as cart
import cartopy.crs as ccrs
# import cmocean.cm as cmo
# import seaborn as sns
from glob import glob

In [ ]:
step = 10
for i in range(1000):
    plt.plot(sim2['lon'][i,:], sim2['lat'][i,:]);

plt.xlabel('Longitude')
plt.ylabel('Latitude')

In [ ]:
step = 10
plt.scatter(sim2['salinity'][:,step], -sim2['z'][:,step])
plt.xlabel('Salinity (PSU)')
plt.ylabel('Depth (m)')
plt.title(sim2['time'][0,step].values)